# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

Подгружаем библиотеки.

In [2]:
data = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

Загружаем датасеты.

In [3]:
data

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
data1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [5]:
data2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


Просмотрев датасеты, принимаю решение что id не нужно.

In [6]:
data = data.drop(['id'], axis=1)
data1 = data1.drop(['id'], axis=1)
data2 = data2.drop(['id'], axis=1)

Убираем все столбцы с id.

## Обучение и проверка модели

### Разбиваем данные на обучающую и валидационную выборки в соотношении 75:25.

In [7]:
features = data.drop(['product'] , axis=1)
target = data['product']

In [8]:
features1 = data1.drop(['product'] , axis=1)
target1 = data1['product']

In [9]:
features2 = data2.drop(['product'] , axis=1)
target2 = data2['product']

In [10]:
features_data, features_valid, target_data, target_valid = train_test_split(features, target, test_size = 0.25, random_state=12345)
print(features_data.shape)
print(features_valid.shape)

(75000, 3)
(25000, 3)


In [11]:
features_data1, features_valid1, target_data1, target_valid1 = train_test_split(features1, target1, test_size = 0.25, random_state=12345)
print(features_data1.shape)
print(features_valid1.shape)

(75000, 3)
(25000, 3)


In [12]:
features_data2, features_valid2, target_data2, target_valid2 = train_test_split(features2, target2, test_size = 0.25, random_state=12345)
print(features_data2.shape)
print(features_valid2.shape)

(75000, 3)
(25000, 3)


Разделили на обучающую и валидационную выборку.

### Обучяем модель и делаем предсказания на валидационной выборке.

In [13]:
model = LinearRegression()
model.fit(features_data, target_data)
predictions_valid = model.predict(features_valid)

Первый регион.

In [14]:
model1 = LinearRegression()
model1.fit(features_data1, target_data1)
predictions_valid1 = model1.predict(features_valid1)

Второй регион.

In [15]:
model2 = LinearRegression()
model2.fit(features_data2, target_data2)
predictions_valid2 = model2.predict(features_valid2)

Третий регион.

### Сохраняем предсказания и правильные ответы на валидационной выборке.

Не совсем понял этот раздел.

### Выводим на экран средний запас предсказанного сырья и RMSE модели.

In [16]:
predictions_valid.mean()

92.59256778438038

In [17]:
rmse = mean_squared_error(target_valid, predictions_valid) ** 0.5
rmse

37.5794217150813

Среднее значение сырья и RMSE модели для первого региона.

In [18]:
predictions_valid1.mean()

68.728546895446

In [19]:
rmse1 = mean_squared_error(target_valid1, predictions_valid1) ** 0.5
rmse1

0.893099286775616

Среднее значение сырья и RMSE модели для второго региона.

In [20]:
predictions_valid2.mean()

94.96504596800489

In [21]:
rmse2 = mean_squared_error(target_valid2, predictions_valid2) ** 0.5
rmse2

40.02970873393434

Среднее значение сырья и RMSE модели для третьего региона.

### Анализируем результаты

По полученным данным лучший определяем что третий регион лучший.

## Подготовка к расчёту прибыли

###  Все ключевые значения для расчётов сохраняем в отдельных переменных.

In [22]:
BUDGET = 10000000000
INCOME = 450000
WELLS = 200

### Рассчитываем достаточный объём сырья для безубыточной разработки новой скважины. Сравниваем полученный объём сырья со средним запасом в каждом регионе.

In [23]:
expenditure = BUDGET / WELLS / INCOME
expenditure

111.11111111111111

Средние показатели скважин 3х регионов: 92.6, 68.7, 94.9. Нам не хватает.

In [24]:
predictions_valid = pd.Series(predictions_valid).reset_index(drop=True)
predicted_sort = predictions_valid.sort_values(ascending=False)
predicted_sort

9317     180.180713
219      176.252213
10015    175.850623
11584    175.658429
23388    173.299686
            ...    
23129     10.209351
9078       8.798531
21875      6.569477
14503      4.980987
20881     -9.367846
Length: 25000, dtype: float64

## Расчёт прибыли и рисков 

### Выбераем скважины с максимальными значениями предсказаний.

In [41]:
def profit(predicted, target):
    predicted = pd.Series(predicted).reset_index(drop=True)
    target = pd.Series(target).reset_index(drop=True)
    predicted_sort = predicted.sort_values(ascending=False)
    quantity = target[predicted_sort.index][:WELLS].sum()
    profit_finish = ((quantity * INCOME) - BUDGET)/ 1000000000

    return profit_finish






In [53]:
print('Предсказанная прибыль первого региона:', profit(predictions_valid, target_valid))

Предсказанная прибыль первого региона: 3.3208260431398524


In [54]:
print('Предсказанная прибыль второго региона:', profit(predictions_valid1, target_valid1))

Предсказанная прибыль второго региона: 2.4150866966815108


In [56]:
print('Предсказанная прибыль второго региона:', profit(predictions_valid2, target_valid2))

Предсказанная прибыль второго региона: 2.7103499635998327


Наибольшая прибыль у регона 1 и 3.

## Считаем риски и прибыль для каждого региона

### Применяем технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [94]:
state = np.random.RandomState(12345)
def bootstrap(predicted, target):
    target = target.reset_index(drop=True)
    values = []

    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        predicted_subsample = predicted[target_subsample.index]
        
        profit1 = profit(predicted_subsample, target_subsample)
        values.append(profit1)
    
    return pd.Series(values)

### Найходим среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [101]:
def analysis(predicted, target):
    values = bootstrap(predicted, target)
    
    mean = pd.Series(values).mean()
    quantil = [values.quantile(0.025), values.quantile(0.975)]
    
    print("Средняя выручка:", mean)
    print("Доверительный интервал:", quantil)
    print("Риски убытка:", (values < 0).mean())

In [102]:
analysis(predictions_valid, target_valid)

Средняя выручка: 0.3938025093488005
Доверительный интервал: [-0.11977787811425389, 0.9093132357352335]
Риски убытка: 0.057


In [103]:
analysis(predictions_valid1, target_valid1)

Средняя выручка: 0.44827528922909887
Доверительный интервал: [0.05027547525874067, 0.8366259808831102]
Риски убытка: 0.013


In [104]:
analysis(predictions_valid2, target_valid2)

Средняя выручка: 0.38252715326889897
Доверительный интервал: [-0.2020650217268754, 0.9298880449099379]
Риски убытка: 0.094


### Вывод

По нашим предворительным подсчетам мы выделяли регион 1 и 3 по их показателям. После применения техники Bootstrap и нахождения распределения прибыли и получения средней прибыли, доверительного интервала и риска убытков, мы понимаем что лучий регион это №2. Средня выручка которого составляет 448 млн., и самые низкие риски убтка.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: модели обучены и проверены
    - [ ]  Данные корректно разбиты на обучающую и валидационную выборки
    - [ ]  Модели обучены, предсказания сделаны
    - [ ]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [ ]  На экране напечатаны результаты
    - [ ]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [ ]  Для всех ключевых значений созданы константы Python
    - [ ]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [ ]  По предыдущему пункту сделаны выводы
    - [ ]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован